In [2]:
!pip install --upgrade --force-reinstall ibm-watson-machine-learning pandas numpy --no-cache-dir -q

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.2/91.2 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.1/62.1 kB 27.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 157.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.0/61.0 kB 248.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.1/1.1 MB 29.9 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 139.6/139.6 kB 297.0 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 165.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 12.2/12.2 MB 241.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.3/18.3 MB 203.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 229.9/229.9 kB 300.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 509.2/509.2 kB 322.4 MB/s eta 0:

In [3]:
from ibm_watson_machine_learning.foundation_models import Model
from ibm_watson_machine_learning import APIClient
import pandas as pd

In [6]:
API_KEY = "g9uIRQ0Wln9QSeFrPm4dgeZncn7PzEEVQ0i2NTvFaxFi"
PROJECT_ID = "8ee78246-49af-4ffd-83a9-d57a0ba9aef3"
REGION = "us-south"

credentials = {
    "url": f"https://{REGION}.ml.cloud.ibm.com",
    "apikey": API_KEY
}

params = {
    "decoding_method": "greedy",
    "max_new_tokens": 50,
    "temperature": 0.2
}

model_id = "ibm/granite-13b-instruct-v2"

model = Model(
    model_id=model_id,
    params=params,
    credentials=credentials,
    project_id=PROJECT_ID
)


/usr/local/lib/python3.11/dist-packages/ibm_watson_machine_learning/foundation_models/utils/utils.py:273: LifecycleWarning: Model 'ibm/granite-13b-instruct-v2' is in deprecated state from 2025-06-18 until 2025-10-15. IDs of alternative models: ibm/granite-3-3-8b-instruct. Further details: https://dataplatform.cloud.ibm.com/docs/content/wsj/analyze-data/fm-model-lifecycle.html?context=wx&audience=wdp
  warnings.warn(default_warning_template.format(


In [7]:

from google.colab import files
uploaded = files.upload()

df = pd.read_csv("legal_docs.csv")
df.head()


Saving legal_docs.csv to legal_docs.csv


,document_id,summary
0,1,This agreement ensures confidentiality between...
1,2,"This contract defines the scope of services, p..."
2,3,This document outlines the partnership and res...
3,4,This service agreement specifies uptime guaran...
4,5,This document is a Memorandum of Understanding...


In [8]:
def generate_prompt(summary):
    return f"""You are a legal assistant. Based on the text, classify the type of legal document.

Example 1:
Text: "This agreement ensures confidentiality between two parties."
Type: NDA

Example 2:
Text: "This contract outlines payment terms for services."
Type: Contract

Example 3:
Text: "This document describes service responsibilities and legal terms."
Type: Service Agreement

Now classify this:
Text: "{summary}"
Type:"""


In [9]:
predicted_types = []

for summary in df["summary"]:
    prompt = generate_prompt(summary)
    try:
        response = model.generate(prompt)
        result = response['results'][0]['generated_text'].strip()
        predicted_types.append(result)
    except Exception as e:
        predicted_types.append("ERROR: " + str(e))

df["predicted_type"] = predicted_types


In [10]:
output_file = "classified_legal_docs.csv"
df.to_csv(output_file, index=False)
print(f"✅ Saved predictions to: {output_file}")

files.download(output_file)


✅ Saved predictions to: classified_legal_docs.csv


<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>